# Third-party software computation model example

## Introduction

This example file demonstrates how to build a RunModel object which enables UQpy to execute models in third-party software. The files necessary to run this example are:
1. The input template - 'abaqus_input.py'
2. The model script - 'abaqus_fire_analysis.py'
3. The output script - 'extract_abaqus_output.py'
4. The script used by the output script - 'abaqus_output_script.py'

Note: To execute the example in this notebook, it is necessary to have access to the finite element solver Abaqus. 

## Description of the model

This example builds and analyzes a finite element model of a beam bearing uniformly distributed load, which is then subjected to fire load.  

## The script:

Import the python modules used in this example, note down the start time and the current directory, which will be used later to save the results.

In [ ]:
from UQpy.RunModel import *
from UQpy.Reliability import TaylorSeries
import time
import os
import pickle
import glob
import matplotlib.pyplot as plt
import numpy as np

calling_directory = os.getcwd()
t = time.time()

### Building the model:

There are two probabilistic input variables, the fire load density and the yield strength. The fire load density is denoted as 'qtd' and the yield strength is denoted as 'fy' in the template input script. These are different from the default variable names used by RunModel, and hence they must be passed in as one of the inputs while building the RunModel object.

In [ ]:
var_names = ['qtd', 'fy']

#### Create the model object

In [ ]:
abaqus_sfe_model = RunModel(model_script='abaqus_fire_analysis.py', input_template='abaqus_input.py',
                            output_script='extract_abaqus_output.py',
                            var_names=var_names, ntasks=1, model_dir='SFE_Example', verbose=True)
print('Example: Created the model object.')

### Instantiate a UQpy TaylorSeries object 

The fire load density is assumed to be uniformly distributed between 50 MJ/m^2 and 450 MJ/m^2. The yield strength is assumed to be normally distributed, with the parameters being: mean = 250 MPa and coefficient of variation of 7%.

In [ ]:
p1 = TaylorSeries(dimension=2, dist_name=['uniform', 'normal'],
                  dist_params=[[50, 400], [250e6, 17.5e4]],
                  n_iter=100, eps=0.01, df_method='forward',
                  corr=np.eye(2), model=abaqus_sfe_model)

### Performing reliability analysis

In [ ]:
x0 = np.array([250, 2.5e8])
p1.form(seed=x0)

### Print the results

In [ ]:
print('Design point in standard normal space: %s' % p1.DesignPoint_U)
print('Design point in original space: %s' % p1.DesignPoint_X)
print('Hasofer-Lind reliability index: %s' % p1.HL_beta)
print('FORM probability of failure: %s' % p1.Prob_FORM)
print('Performance function values: %s' % p1.g_record)
print('FORM x points: %s' % p1.x_record)
print('FORM u points: %s' % p1.u_record)
print('Iterations: %s' % p1.iterations)

### Save the results

In [ ]:
results = {'outputs': [p1.DesignPoint_U, p1.DesignPoint_X, p1.HL_beta, p1.Prob_FORM, p1.iterations,
                       p1.g_record, p1.dg_record, p1.u_check, p1.g_check, p1.x_record, p1.u_record, p1.alpha_record]}

# Pickle the results dictionary in the current directory. The basename and extension of the desired pickle file:
res_basename = 'FORM_results'
res_extension = '.pkl'

# Create a new results file with a larger index than any existing results files with the same name in the current
# directory.
res_file_list = glob.glob(res_basename + '_???' + res_extension)
if len(res_file_list) == 0:
    res_file_name = res_basename + '_000' + res_extension
else:
    max_number = max(res_file_list).split('.')[0].split('_')[-1]
    res_file_name = res_basename + '_%03d' % (int(max_number) + 1) + res_extension

res_file_name = os.path.join(calling_directory, res_file_name)
# Save the results to this new file.
with open(res_file_name, 'wb') as f:
    pickle.dump(results, f)
print('Saved the results to ' + res_file_name)


print('Example: Done!')
print('Time elapsed: %.2f minutes' % float((time.time() - t) / 60.0))